# Homework 1.3

In [ ]:
from heapq import heappush, heappop
from collections import defaultdict
import math


class Graph:
    def __init__(self, undirected=True):
        self.adj = defaultdict(dict)
        self.undirected = undirected

    def add_edge(self, u, v, w):
        self.adj[u][v] = w
        if self.undirected:
            self.adj[v][u] = w

    def neighbors(self, u):
        return self.adj[u].items()


def a_star(G, start, goal, h):
    g = defaultdict(lambda: math.inf) 
    g[start] = 0.0
    prev = {}                           
    pq = []                           
    heappush(pq, (h[start], start))
    seen = set()

    while pq:
        _, u = heappop(pq)
        if u in seen: 
            continue
        seen.add(u)
        if u == goal:
            break

        for v, w in G.neighbors(u):
            alt = g[u] + w
            if alt < g[v]:
                g[v] = alt
                prev[v] = u
                heappush(pq, (alt + h[v], v))

    return g, prev

def reconstruct_path(prev, start, goal):
    if start == goal:
        return [start]
    if goal not in prev:
        return []
    path = [goal]
    while path[-1] != start:
        path.append(prev[path[-1]])
    return list(reversed(path))

def routing_table(prev, dist, start):
    table = {}
    for node, d in dist.items():
        if d is math.inf:
            continue
        cur = node
        while cur in prev and prev[cur] != start:
            cur = prev[cur]
        table[node] = (d, start if node == start else cur)
    return dict(sorted(table.items()))


def build_1_graph():
    """
    Matches the LEFT picture in the slide.
    Edges (undirected) and orange heuristic values.
    """
    G = Graph()
    edges = [
        ("A","B",9),
        ("A","C",4),
        ("A","D",7),
        ("B","E",11),
        ("C","E",17),
        ("C","F",12),
        ("D","F",14),
        ("D","C",18),
        ("F","Z",9),
        ("E","Z",5),
    ]
    for u,v,w in edges:
        G.add_edge(u,v,w)

    H = {"A":21, "B":14, "C":18, "D":18, "E":5, "F":8, "Z":0}

    return G, H


def build_2_graph():
    """
    Matches the RIGHT picture in the slide.
    Edges (undirected) and orange heuristic values.
    """
    G = Graph()
    edges = [
        ("A","B",5),
        ("A","C",4),
        ("B","C",1),
        ("B","D",5),
        ("C","D",8),
        ("D","E",2),
        ("C","E",10),
        ("D","Z",6),
        ("E","Z",5),
    ]
    for u,v,w in edges:
        G.add_edge(u,v,w)

    H = {"A":11, "B":8, "C":8, "D":4, "E":2, "Z":0}

    return G, H


def run_case(title, build_fn):
    G, H = build_fn()
    dist, prev = a_star(G, "A", "Z", H)
    path = reconstruct_path(prev, "A", "Z")
    table = routing_table(prev, dist, "A")

    print(f"\n{title}")
    print("Shortest path A->Z:", path)
    print("Total cost:", dist["Z"])
    print("Routing table from A:")
    for node, (cost, next_hop) in table.items():
        print(f"  to {node:>2}: cost={int(cost) if cost.is_integer() else cost}, next_hop={next_hop}")

if __name__ == "__main__":
    run_case("HW 1.3 (1 Graph)", build_1_graph)
    run_case("HW 1.3 (2 Graph)", build_2_graph)



--- HW 1.3 (1 graph) ---
Shortest path A->Z: ['A', 'C', 'F', 'Z']
Total cost: 25.0
Routing table from A:
  to  A: cost=0, next_hop=A
  to  B: cost=9, next_hop=B
  to  C: cost=4, next_hop=C
  to  D: cost=7, next_hop=D
  to  E: cost=20, next_hop=B
  to  F: cost=16, next_hop=C
  to  Z: cost=25, next_hop=C

--- HW 1.3 (2 graph) ---
Shortest path A->Z: ['A', 'B', 'D', 'Z']
Total cost: 16.0
Routing table from A:
  to  A: cost=0, next_hop=A
  to  B: cost=5, next_hop=B
  to  C: cost=4, next_hop=C
  to  D: cost=10, next_hop=B
  to  E: cost=12, next_hop=B
  to  Z: cost=16, next_hop=B


# Homework 2.3

In [ ]:
from heapq import heappush, heappop
from collections import defaultdict
import math

class Graph:
    def __init__(self, undirected=True):
        self.adj = defaultdict(dict)
        self.undirected = undirected
    def add_edge(self, u, v, w):
        self.adj[u][v] = w
        if self.undirected:
            self.adj[v][u] = w
    def neighbors(self, u):
        return self.adj[u].items()

def dijkstra(G, start):
    dist = defaultdict(lambda: math.inf)
    prev = {}
    dist[start] = 0.0
    pq = [(0.0, start)]
    seen = set()

    while pq:
        d, u = heappop(pq)
        if u in seen: 
            continue
        seen.add(u)
        for v, w in G.neighbors(u):
            alt = d + w
            if alt < dist[v]:
                dist[v] = alt
                prev[v] = u
                heappush(pq, (alt, v))
    return dist, prev

def reconstruct_path(prev, start, goal):
    if start == goal: 
        return [start]
    if goal not in prev: 
        return []
    path = [goal]
    while path[-1] != start:
        path.append(prev[path[-1]])
    return path[::-1]

def routing_table(prev, dist, start):
    table = {}
    for node, d in dist.items():
        if d is math.inf: 
            continue
        cur = node
        while cur in prev and prev[cur] != start:
            cur = prev[cur]
        table[node] = {"cost": d, "next_hop": (start if node == start else cur)}
    return dict(sorted(table.items()))


def build_1_graph():
    G = Graph()
    edges = [
        ("A","B",4), ("A","C",3),
        ("C","B",1),
        ("B","D",5), ("B","E",12), ("B","F",5),
        ("C","D",7), ("C","E",10),
        ("D","E",2),
        ("E","Z",5),
        ("F","Z",16),
    ]
    for u,v,w in edges: G.add_edge(u,v,w)
    return G


def build_2_graph():
    G = Graph()
    edges = [
        ("A","B",9), ("A","C",4), ("A","D",7),
        ("B","E",11),
        ("C","E",17), ("C","F",12),
        ("D","F",14), ("D","C",18),
        ("F","Z",9), ("E","Z",5),
    ]
    for u,v,w in edges: G.add_edge(u,v,w)
    return G

def run_case(title, build_fn):
    G = build_fn()
    dist, prev = dijkstra(G, "A")
    path = reconstruct_path(prev, "A", "Z")
    print(f"\n{title}")
    print("Shortest path A→Z:", path)
    print("Length:", dist["Z"])
    print("Routing table from A:")
    for node, info in routing_table(prev, dist, "A").items():
        print(f"  {node}: cost={int(info['cost']) if info['cost'].is_integer() else info['cost']}, next_hop={info['next_hop']}")

if __name__ == "__main__":
    run_case("HW 2.3 (1 Graph)", build_1_graph)
    run_case("HW 2.3 (2 Graph)", build_2_graph)



--- HW 2.3 (1 Graph) ---
Shortest path A→Z: ['A', 'B', 'D', 'E', 'Z']
Length: 16.0
Routing table from A:
  A: cost=0, next_hop=A
  B: cost=4, next_hop=B
  C: cost=3, next_hop=C
  D: cost=9, next_hop=B
  E: cost=11, next_hop=B
  F: cost=9, next_hop=B
  Z: cost=16, next_hop=B

--- HW 2.3 (2 Graph) ---
Shortest path A→Z: ['A', 'C', 'F', 'Z']
Length: 25.0
Routing table from A:
  A: cost=0, next_hop=A
  B: cost=9, next_hop=B
  C: cost=4, next_hop=C
  D: cost=7, next_hop=D
  E: cost=20, next_hop=B
  F: cost=16, next_hop=C
  Z: cost=25, next_hop=C


# Homework 3

In [ ]:
from heapq import heappush, heappop
from collections import defaultdict

class Graph:
    def __init__(self, undirected=True):
        self.adj = defaultdict(dict); self.undirected = undirected
    def add_edge(self, u, v, w):
        self.adj[u][v] = w
        if self.undirected: self.adj[v][u] = w
    def neighbors(self, u): 
        return self.adj[u].items()

def reconstruct(prev, s, t):
    if t not in prev and s != t: 
        return []
    p=[t]
    while p[-1]!=s: p.append(prev[p[-1]])
    return p[::-1]

def greedy_best_first(G, start, goal, h):
    prev = {}; seen=set(); order=[]; pq=[]
    heappush(pq,(h(start), start))
    while pq:
        _, u = heappop(pq)
        if u in seen: 
            continue
        seen.add(u); order.append(u)
        if u == goal: 
            return order, reconstruct(prev, start, goal)
        for v,_ in G.neighbors(u):
            if v not in seen:
                prev[v]=u; heappush(pq,(h(v), v))
    return order, []

def build_graph():
    G = Graph()
    edges = [
        ("A","B",4), ("A","C",2),
        ("C","B",1), ("C","D",8), ("C","E",10),
        ("B","D",5), ("D","E",2), ("D","Z",6),
        ("E","Z",5),
    ]
    for u,v,w in edges: G.add_edge(u,v,w)
    return G

def path_cost(G, path):
    if not path: 
        return None
    c=0
    for i in range(len(path)-1): c += G.adj[path[i]][path[i+1]]
    return c

if __name__ == "__main__":
    G = build_graph()
    H = {"A":9,"B":7,"C":8,"D":2,"E":3,"Z":0}
    order, path = greedy_best_first(G, "A", "Z", lambda n: H[n])
    print("Visited order:", order)
    print("Path:", path)
    print("Cost:", path_cost(G, path))


Visited order: ['A', 'B', 'D', 'Z']
Path: ['A', 'B', 'D', 'Z']
Cost: 15


# Homework 4

In [4]:
class Graph:
    def __init__(self, undirected=True):
        from collections import defaultdict
        self.adj = defaultdict(dict); self.undirected = undirected
    def add_edge(self, u, v, w):
        self.adj[u][v] = w
        if self.undirected: self.adj[v][u] = w
    def neighbors(self, u): return self.adj[u].items()

def all_paths_with_costs(G, start, goal):
    res=[]
    def dfs(u, cost, path, used):
        if u==goal:
            res.append((path[:], cost)); return
        for v,w in G.neighbors(u):
            if v in used: continue
            used.add(v); path.append(v)
            dfs(v, cost+w, path, used)
            path.pop(); used.remove(v)
    dfs(start, 0, [start], {start})
    return res

def build_graph():
    G = Graph()
    edges = [
        ("A","B",4), ("A","C",2),
        ("C","B",1), ("C","D",8), ("C","E",10),
        ("B","D",5), ("D","E",2), ("D","Z",6),
        ("E","Z",5),
    ]
    for u,v,w in edges: G.add_edge(u,v,w)
    return G

if __name__ == "__main__":
    G = build_graph()
    paths = all_paths_with_costs(G, "A", "Z")
    paths.sort(key=lambda x: x[1])
    print("Total paths:", len(paths))
    for p,c in paths:
        print(p, "cost", c)


Total paths: 13
['A', 'C', 'B', 'D', 'Z'] cost 14
['A', 'B', 'D', 'Z'] cost 15
['A', 'C', 'B', 'D', 'E', 'Z'] cost 15
['A', 'B', 'D', 'E', 'Z'] cost 16
['A', 'C', 'D', 'Z'] cost 16
['A', 'C', 'D', 'E', 'Z'] cost 17
['A', 'C', 'E', 'Z'] cost 17
['A', 'B', 'C', 'D', 'Z'] cost 19
['A', 'B', 'C', 'D', 'E', 'Z'] cost 20
['A', 'B', 'C', 'E', 'Z'] cost 20
['A', 'C', 'E', 'D', 'Z'] cost 20
['A', 'B', 'C', 'E', 'D', 'Z'] cost 23
['A', 'B', 'D', 'C', 'E', 'Z'] cost 32


# Homework 5

In [5]:
import time

N = 9; BOX = 3

def is_valid(b, r, c, k):
    if any(b[r][j]==k for j in range(N)): return False
    if any(b[i][c]==k for i in range(N)): return False
    br, bc = (r//BOX)*BOX, (c//BOX)*BOX
    for i in range(BOX):
        for j in range(BOX):
            if b[br+i][bc+j]==k: return False
    return True

def find_empty(b):
    for i in range(N):
        for j in range(N):
            if b[i][j]==0: return i,j
    return None

def solve(b):
    pos = find_empty(b)
    if not pos: return True
    r,c = pos
    for k in range(1,10):
        if is_valid(b, r, c, k):
            b[r][c]=k
            if solve(b): return True
            b[r][c]=0
    return False

def solve_with_timing(board):
    import copy
    b = copy.deepcopy(board)
    t0 = time.perf_counter()
    ok = solve(b)
    dt = time.perf_counter() - t0
    return ok, b, dt

def pretty(b):
    return "\n".join(" ".join(str(x) if x else "." for x in row) for row in b)

PUZZLES = {
    "easy": [[
        [5,3,0,0,7,0,0,0,0],
        [6,0,0,1,9,5,0,0,0],
        [0,9,8,0,0,0,0,6,0],
        [8,0,0,0,6,0,0,0,3],
        [4,0,0,8,0,3,0,0,1],
        [7,0,0,0,2,0,0,0,6],
        [0,6,0,0,0,0,2,8,0],
        [0,0,0,4,1,9,0,0,5],
        [0,0,0,0,8,0,0,7,9],
    ]],
    "medium": [],
    "hard": [],
    "expert": [],
}

if __name__ == "__main__":
    board = [row[:] for row in PUZZLES["easy"][0]]
    print("Input:\n", pretty(board))
    ok, solved, dt = solve_with_timing(board)
    print("\nSolved:", ok, f"in {dt:.4f}s")
    print(pretty(solved))


Input:
 5 3 . . 7 . . . .
6 . . 1 9 5 . . .
. 9 8 . . . . 6 .
8 . . . 6 . . . 3
4 . . 8 . 3 . . 1
7 . . . 2 . . . 6
. 6 . . . . 2 8 .
. . . 4 1 9 . . 5
. . . . 8 . . 7 9

Solved: True in 0.0339s
5 3 4 6 7 8 9 1 2
6 7 2 1 9 5 3 4 8
1 9 8 3 4 2 5 6 7
8 5 9 7 6 1 4 2 3
4 2 6 8 5 3 7 9 1
7 1 3 9 2 4 8 5 6
9 6 1 5 3 7 2 8 4
2 8 7 4 1 9 6 3 5
3 4 5 2 8 6 1 7 9
